This notebook generates detector plane images (dpis) of confirmed SWIFT Burst Alert Telescope (BAT) gamma ray bursts (GRBs) as posted by swift.gsfc.nasa.gov.

The GRB time intervals are extracted using their event data provided by the website and are centered using a 64ms time window. The DPI is constructed from the DETX and DETY coordinates of each detected event.

Two resolutions are available, corresponding to two support vector machine models:

"original" : full detector plane resolution (286 x 173)

"lowRES" : binned resolution (~ 16 x 16)

In [35]:
resolution = "lowRES"  # Choose between "original" or "lowRES"
total = 250  # Number of dpi files to save

In [36]:
import os, io, requests
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from astropy.table import Table
from astropy.io import fits

### helper functions

In [37]:
base_url = "https://swift.gsfc.nasa.gov/results/batgrbcat/"

grbdpi_folder = "GRBdpi" if resolution == "original" else "GRBdpi_lowRES"
os.makedirs(grbdpi_folder, exist_ok=True)

def get_grb_list():
    response = requests.get(base_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table')
        if not table:
            print("No GRB table found.")
            return []
    
    grbs = []
    for row in table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) > 2:
            name = cols[0].text.strip()
            trigid = cols[1].text.strip()
            if name and trigid.isdigit():
                grbs.append((name, trigid))
    
    return grbs
    
    
def det2dpi_sands_ebins(tab, ebins, weights=None):

    detxs_by_sand0 = np.arange(0, 286 - 15, 18)
    detxs_by_sand1 = detxs_by_sand0 + 15
    xbins = np.append(detxs_by_sand0, [detxs_by_sand1[-1]])

    detys_by_sand0 = np.arange(0, 173 - 7, 11)
    detys_by_sand1 = detys_by_sand0 + 7
    ybins = np.append(detys_by_sand0, [detys_by_sand1[-1]])
    
    sand_dpis = np.swapaxes(
        np.histogramdd([tab["DETX"], tab["DETY"], tab['ENERGY']], bins=[xbins, ybins, ebins], weights=weights)[
            0
        ],
        0,
        1,
    )

    return sand_dpis


def gettrig(trigger_id):      # the path to an event file from the base url includes
    tid_str = str(trigger_id) # three zeros after the tigger ID, and as many preceding
    padded = tid_str + '000'  # the ID to total 11 digits in the event file name
    trigstring = padded.zfill(11)  # *excludes those with differing trigger IDs*
    return trigstring


def process_dpis(event_table, index, resolution, dpi_path):
    global saved_dpis
    

    if 'DETX' in event_table.columns and 'DETY' in event_table.columns:
        if event_table['DETX'].size == 0 or event_table['DETY'].size == 0:
            print(f"Warning: Empty DETX/DETY for GRB {index}")
            return

        if resolution == "original":
            xbins = np.arange(286 + 1) - 0.5
            ybins = np.arange(173 + 1) - 0.5
            dpi = plt.hist2d(event_table['DETX'], event_table['DETY'], bins=[xbins, ybins], vmin=0, vmax=2)[0]
            plt.close()
        else:
            ebins = [15, 50, 350]
            dpi = det2dpi_sands_ebins(event_table, ebins)

        np.save(dpi_path, dpi)
        print(f"Saved {dpi_path}")
        saved_dpis += 1
    else:
        print(f"Missing DETX/DETY columns for GRB {index}")

        
def process_grb(grb_name, trigger_id, index, resolution, total, grbdpi_folder):
    global saved_dpis
    
    dpi_filename = f"GRBdpi{'_lowRES' if resolution == 'lowRES' else ''}_{saved_dpis + 1}.npy"
    dpi_path = os.path.join(grbdpi_folder, dpi_filename)
    
    if os.path.exists(dpi_path):
        #print(f"Skipping GRB {grb_name} (dpi {index}) because the DPI file already exists.")
        saved_dpis += 1
        return
    
    try: 
        trigstring = gettrig(trigger_id)
        event_url = f"{base_url}{grb_name}/data_product/{trigstring}/bat/event/sw{trigstring}bevshsp_uf.evt"

        response = requests.get(event_url)
        response.raise_for_status()
    
        with fits.open(io.BytesIO(response.content)) as hdul:
            trig_time = hdul[1].header['TRIGTIME']
            event_table = Table(hdul[1].data)
            
            event_table = event_table[(event_table['TIME'] >= trig_time) & 
                                      (event_table['TIME'] <= trig_time + 0.064)]  # 64 ms dpi
            
            print(f"Number of events in 64ms window: {len(event_table)}")
            process_dpis(event_table, index, resolution, dpi_path)
            
    except Exception as e:
        e
        #print(f"Skipping {grb_name}: {e}")

In [38]:
def MyMain(resolution="original", total=300):   # default values
    global saved_dpis
    saved_dpis = 0
    assert resolution in ["original", "lowRES"], "Resolution must be 'original' or 'lowRES'"
     
    grbdpi_folder = "GRBdpi" if resolution == "original" else "GRBdpi_lowRES"
    os.makedirs(grbdpi_folder, exist_ok=True)
        
    if saved_dpis >= total:
        #print(f"{saved_dpis} GRB DPI files already exist in '{grbdpi_folder}'. Done.")
        return
    
    grbs = get_grb_list()
    
    for i, (name, trigid) in enumerate(grbs, start=1):
        if saved_dpis >= total:
            break

        #print(f"\tProcessing GRB {name} (index {i})...")  
        process_grb(name, trigid, i, resolution, total, grbdpi_folder)
    
    print(f"Processing complete. {saved_dpis} GRB dpi files saved.")


In [39]:
MyMain(resolution=resolution, total=total)

Processing complete. 250 GRB dpi files saved.
